# 入门指南

## Tensors

In [2]:
from __future__ import print_function
import torch

In [3]:
x = torch.rand(5, 3)
print(x)


 0.1534  0.8068  0.3515
 0.8412  0.2218  0.6644
 0.8557  0.4681  0.1206
 0.1047  0.8225  0.3997
 0.4232  0.5157  0.9953
[torch.FloatTensor of size 5x3]



In [4]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

AttributeError: module 'torch' has no attribute 'long'

In [10]:
#直接从数据构建tensor
x = torch.tensor([5.5, 3])
print(x)

tensor([ 5.5000,  3.0000])


In [11]:
# 或者在现有张量的基础上建立一个张量。这些方法将重用输入张量的属性
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                                      # result has the same size

tensor([[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]], dtype=torch.float64)
tensor([[-0.0415, -0.1703, -1.4125],
        [ 0.2924,  0.0202,  1.0450],
        [ 1.6637,  1.0256,  0.3090],
        [ 0.0571, -0.5585,  0.8253],
        [-0.5552, -2.0102, -0.5302]])


In [3]:
x = torch.Tensor(5, 3)
print(x)

tensor([[-2.6553e-26,  4.5877e-41, -2.6553e-26],
        [ 4.5877e-41,         nan,  9.8091e-45],
        [ 6.4174e-10,  2.1240e+20,  2.0801e+23],
        [ 1.6689e-07,  6.7410e+22,  1.7089e-04],
        [ 1.7394e-04,  2.1277e+23,  5.0948e-14]])


In [4]:
print(x.size())

torch.Size([5, 3])


torch.Size实际上是一个元组，因此它支持所有的tuple操作。

## 操作
张量上的操作有多重语法形式，下面我们以加法为例进行讲解。

### 语法1

In [5]:
y = torch.rand(5,3)
print(x+y)

tensor([[ 2.0946e-01,  4.5526e-01,  9.3376e-02],
        [ 8.6602e-01,         nan,  8.2369e-01],
        [ 3.2159e-01,  2.1240e+20,  2.0801e+23],
        [ 3.9475e-01,  6.7410e+22,  1.4715e-01],
        [ 3.2186e-01,  2.1277e+23,  4.6093e-01]])


### 语法2

In [ ]:
print(torch.add(x,y))

### 语法3：给出一个输出向量

In [7]:
result = torch.Tensor(5,3)
torch.add(x, y, out=result)
print(result)

tensor([[ 2.0946e-01,  4.5526e-01,  9.3376e-02],
        [ 8.6602e-01,         nan,  8.2369e-01],
        [ 3.2159e-01,  2.1240e+20,  2.0801e+23],
        [ 3.9475e-01,  6.7410e+22,  1.4715e-01],
        [ 3.2186e-01,  2.1277e+23,  4.6093e-01]])


### 语法4：原地操作（in-place）

In [25]:
# 把x加到y上
y.add_(x)
print(y)

tensor([-0.9139], device='cuda:0')


注意

任何在原地(in-place)改变张量的操作都有一个`_`后缀。例如`x.copy_(y)`, `x.t_()`操作将改变x.


你可以使用所有的numpy索引操作。

In [1]:
print(x[:,1])

NameError: name 'x' is not defined

调整大小：如果你想要调整/重塑张量，你可以使用`torch.view`：

In [27]:
x = torch.randn(4,4)
y = x.view(16)
z = x.view(-1,8) #-1处的值是从其他维度推断出来的
print(x.size(),y.size(),z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


如果你有一个单元素tensor，使用`.item（）`来获得作为Python数的值

In [16]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([-1.9139])
-1.9139426946640015


[这里](https://pytorch.org/docs/stable/torch.html)描述了一百多种张量操作，包括转置，索引，数学运算，线性代数，随机数等

## Numpy桥
- 将一个 Torch Tensor转换成一个NumPy矩阵，反之亦然。
- Torch张量和NumPy矩阵将共享它们的底层内存位置，改变一个则会改变另一个。
- CPU上所有的张量，除了char张量外都支持NumPy和Tensor之间的相互转换。

### 将一个 Torch Tensor转换成一个NumPy数组

In [17]:
a = torch.ones(5)
print(a)

tensor([ 1.,  1.,  1.,  1.,  1.])


In [18]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


In [19]:
a.add_(1)
print(a)
print(b)

tensor([ 2.,  2.,  2.,  2.,  2.])
[2. 2. 2. 2. 2.]


### 将一个NumPy数组转换成一个Torch Tensor

In [22]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([ 2.,  2.,  2.,  2.,  2.], dtype=torch.float64)


## CUDA Tensors
使用`.to`方法可以将张量移动到任何设备上。

In [23]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([-0.9139], device='cuda:0')
tensor([-0.9139], dtype=torch.float64)
